# Mô tả bài toán

## Input
Dữ liệu bao gồm các ảnh trắng đen (có 1 kênh màu) được cấu trúc và xử lý theo ma trận cột, trong đó mỗi cột là một ảnh và 28x28 dòng cho mỗi pixel. Các tầng khác nhau có phương thức foward tương ứng với hành vi của tầng.

## Output
Dữ liệu có cấu trúc giống với dữ liệu output nhưng số lượng dòng của ma trận là bội số theo channel đầu ra. 

# Mô tả tuần tự


# Mô tả song song

## Phân tích
Phân tích được thực hiện dựa trên quy trình sau:
**Input** (tên biến, vị trí của biến, kích thước cụ thể) &rarr; **Output** &rarr; **Filter** &rarr; **Truyền dữ liệu** cho device; đồ hình của device &rarr; **Tính toán**.


1. Input
* Ở tầng C1 và C3: dữ liệu đầu vào là ma trận `bottom` ở file `/src/layer/conv.cc`. Ma trận gồm các ảnh trắng đen (có 1 kênh màu) được cấu trúc và xử lý theo ma trận cột, trong đó mỗi cột là một ảnh và mỗi dòng là mỗi pixel của ảnh tương ứng.
* Tầng C1: `bottom` có kích thước (28x28, 60000).
* Tầng C3: `bottom` có kích thước (12x12x6, 60000), là dữ liệu đầu ra ở tầng P2 sau khi đã pooling.
2. Output
* Ở tầng C1 và C3: dữ liệu đầu ra là ma trận `top`. Ma trận gồm các ảnh trắng đen (có 1 kênh màu) được cấu trúc và xử lý theo ma trận cột, trong đó mỗi cột là một ảnh và mỗi dòng là mỗi pixel của ảnh tương ứng.
* Tầng C1: `top` có kích thước (24x24x6, 60000), với 6 là số lượng filter.
* Tầng C3: `top` có kích thước (8x8x16, 60000), với 16 là số lượng filter.
3. Filter (từ file /src/layer/conv.cc)
* filter là ma trận `weight` có kích thước (channel_in * h_kernel * w_kernel, channel_out). Giá trị của filter có kiểu float.
4. Bias (từ file /src/layer/conv.cc)
* bias cho quá trình forward là vector `bias` có kích thước (channel_out). Giá trị của bias có kiểu float.


## Thiết kế
### Filter 1



## Cài đặt


## Đánh giá kết quả


# Thực hiện

## 1. Thay đổi mô hình của tác giả thành mô hình gợi ý
Cần lưu ý ở file demo.cc trước khi chạy: dataset - model - epoch.

Chỉnh sửa ở file demo.cc:
* Thêm thư viện time.h và 2 đối tượng time_t: start + end để đo thời gian train.
* Convolutional layer (Conv2D) có filter 5x5, stride = 1x1, padding = 0, số lượng là 6 và 16 tương ứng với mô hình.
* Pooling layer (MaxPooling2D) có kích thước là 2x2, stride = 2x2, padding = 0.

## 2. Setup cuda và cmake

Ý tưởng ban đầu là đổi file /src/layer/conv.cc thành /src/layer/conv.cu và cài đặt các filter ở đây nhưng không thể tìm ra giải pháp.

1. Thay đổi compiler của cmake để biên dịch cuda [(tham khảo)](https://dominoc925.blogspot.com/2021/03/creating-simple-cuda-with-cmake-c.html)
* Chỉnh sửa hai dòng ở file /CMakeLists.txt.
* Sửa dòng số 42 ở /third_party/eigen/Eigen/Core: math_functions.h

2. Cải tiến để chương trình có thể nhận cmd argument:
* Thêm thuộc tính public `int use_device` ở file /src/network.h và /src/layer.h
* Ý tưởng: network sẽ nhận yêu cầu sử dụng filter (GPU) từ người dùng thông qua cmd với 0 = sử dụng cpu (convolution tuần tự), k (k>0) = filter loại k. Phương thức forward của network sẽ setup `use_device` cho các layer. Giá trị `use device` chỉ được sử dụng ở file `conv.cc` để lựa chọn filter.
* Ở file `demo.cc`: setup giá trị `use_device` cho dnn (sử dụng atoi() vì cuda 12.2 không hỗ trợ stoi()).


## 3. Sử dụng eigen với cuda và cmake

* Chỉnh sửa 3 file: filter.h, filter.cu, conv.cc.
* Truyền dữ liệu từ file conv.cc (Matrix) sang file filter.cu: sử dụng con trỏ `float*` để lưu trữ dữ liệu dưới dạng column-major (dạng mặc định của eigen). Để kiểm tra quá trình truyền dữ liệu, có thể in ra từng phần tử của con trỏ (không sử dụng sizeof() vì sẽ làm hủy chương trình. Chưa tìm ra nguyên nhân cho trường hợp này).
* Thêm `#ifndef USE_CUDA` + `#endif` ở file filter.cu.



Nguồn:
1. [How to work with Eigen in CUDA kernels](https://stackoverflow.com/questions/23802209/how-to-work-with-eigen-in-cuda-kernels)
2. [Convert Eigen Matrix to C array](https://stackoverflow.com/questions/8443102/convert-eigen-matrix-to-c-array)

## 4. Thực thi với cuda filter



# KẾ HOẠCH THỰC HIỆN
20-26/11: tìm hiểu về cnn, LeNet-5, setup và chạy mã nguồn mini-dnn-cpp với bộ dữ liệu mnist.

27/11 - 03/12: tìm hiểu mã nguồn mini-dnn-cpp, thay đổi mô hình với cấu trúc LeNet-5 (được tinh chỉnh theo yêu cầu của đồ án) và chạy mô hình trên tập dữ liệu fashion MNIST.

08/12 - 27/12: cài đặt GPU Convolution kernel cơ bản.



# NGUỒN THAM KHẢO
BASE MODEL: mini-dnn-cpp
* https://github.com/iamhankai/mini-dnn-cpp.git

DATASET: fashion MNIST
* https://github.com/zalandoresearch/fashion-mnist

RELATED KNOWLEDGE: CNN + LeNet-5
* https://www.kaggle.com/code/blurredmachine/lenet-architecture-a-complete-guide/notebook
* https://setosa.io/ev/image-kernels/
* https://poloclub.github.io/cnn-explainer/
* https://www.tensorflow.org/
*
